In [81]:
'''
设置全局的变量
导入包
'''
import tushare as ts
import datetime
from dateutil.relativedelta import relativedelta

# 全局的业务变量
class BusinessVal:
    # 标的资产，暂时只支持标的资产是一种指数
    underlyingAsset = '000001.SH'
    # 敲出比例
    knockOutRatio = 1.00
    # 敲入比例
    knockInRatio = 0.80
    # 期限，单位是月
    timeLimit = 12
    # start_date
    startDate = '20100101'
    # end_date
    endDate='20200522'
    # 月利率
    monthlyRate = 0.1
    # 保证金
    deposit = 20
    # 名义本金倍数，名义本金除以保证金
    multiple = 5

# 全局的技术变量
class TechVal:
    # tushare_token
    tushareToken = '19280d5f247ae49caae39e24a4363308c5396514275535ed770716c2'

# 初始化 tushare
ts.set_token(TechVal.tushareToken)
pro = ts.pro_api()

print('标的资产   ' + BusinessVal.underlyingAsset)
print('敲出比例   ' + str(BusinessVal.knockOutRatio))
print('敲入比例   ' + str(BusinessVal.knockInRatio))
print('期限   ' + str(BusinessVal.timeLimit))
print('回测起始时间   ' + BusinessVal.startDate)
print('回测终止时间   ' + BusinessVal.endDate)


标的资产   000001.SH
敲出比例   1.0
敲入比例   0.8
期限   12
回测起始时间   20100101
回测终止时间   20200522


In [82]:
'''
获取行情数据
'''
quote = pro.index_daily(ts_code=BusinessVal.underlyingAsset, start_date = BusinessVal.startDate, end_date=BusinessVal.endDate)
# 行反转
quote = quote.reindex(index=quote.index[::-1])
print(quote)


        ts_code trade_date      close       open       high        low  \
2522  000001.SH   20100104  3243.7600  3289.7500  3295.2790  3243.3190   
2521  000001.SH   20100105  3282.1790  3254.4680  3290.5120  3221.4620   
2520  000001.SH   20100106  3254.2150  3277.5170  3295.8680  3253.0440   
2519  000001.SH   20100107  3192.7760  3253.9910  3268.8190  3176.7070   
2518  000001.SH   20100108  3195.9970  3177.2590  3198.9200  3149.0170   
...         ...        ...        ...        ...        ...        ...   
4     000001.SH   20200518  2875.4176  2872.5244  2889.9795  2862.2709   
3     000001.SH   20200519  2898.5760  2897.6867  2900.2187  2887.5768   
2     000001.SH   20200520  2883.7378  2896.4657  2896.4657  2876.1766   
1     000001.SH   20200521  2867.9237  2890.7213  2891.5177  2864.2073   
0     000001.SH   20200522  2813.7654  2863.0462  2863.0462  2808.0177   

      pre_close   change  pct_chg          vol        amount  
2522  3277.1390 -33.3790  -1.0185  109447927.0  

In [83]:
'''
定义一个函数，获取最近的一个交易日
'''
# 交易日历
tradeCalendar = pro.trade_cal(is_open='1')
print(tradeCalendar)

def getLeastTradeDate(searchData):
    '''
    获取searchData的在未来最近一个交易日
    :param searchData: 传入日期
    :return: outDate
    '''
    
    # 如果结果是空，说明不是交易日
    while (tradeCalendar.loc[tradeCalendar['cal_date'] == searchData]).empty:
        searchData = (datetime.datetime.strptime(searchData,'%Y%m%d') + relativedelta(days=1)).strftime('%Y%m%d')
    
    rntDate = searchData
    return rntDate

def isTradeDate(checkDate):
    '''
    判断checkData是否是交易日
    :param checkDate: 
    :return: 
    '''
    # 如果结果是空，说明不是交易日
    if (tradeCalendar.loc[tradeCalendar['cal_date'] == checkDate]).empty:
        return False
    else:
        return True
    

     exchange  cal_date  is_open
0         SSE  19901219        1
1         SSE  19901220        1
2         SSE  19901221        1
3         SSE  19901224        1
4         SSE  19901225        1
...       ...       ...      ...
7338      SSE  20201225        1
7339      SSE  20201228        1
7340      SSE  20201229        1
7341      SSE  20201230        1
7342      SSE  20201231        1

[7343 rows x 3 columns]


In [84]:
# 模拟
def simulate(anchorDate,anchorPrice,knockOutPrice,knockInPrice,observeList):
    '''
    模拟
    :param anchorDate: 锚定日期
    :param anchorPrice: 锚定价格
    :param knockOutPrice: 敲出价格
    :param knockInPrice: 敲入价格
    :param observeList: 观察日列表
    :return: 生成word文档
    '''
    # 是否曾经敲入过
    isKnockIn = False
    # 是否曾经敲出过
    isKnockOut = False
    # 模拟日期
    simulateDate = anchorDate
    # 如果没到期
    while datetime.datetime.strptime(simulateDate,'%Y%m%d') <= datetime.datetime.strptime(observeList[-1],'%Y%m%d'):
        # 如果不是交易日
        if not isTradeDate(checkDate=simulateDate):
            simulateDate = (datetime.datetime.strptime(simulateDate,'%Y%m%d') + relativedelta(days=1)).strftime('%Y%m%d')
            continue
        # 如果是交易日
        # 获取行情数据
        simulateQuote = quote.loc[quote['trade_date'] == simulateDate]
        # 判断是否敲入
        if simulateQuote['close'].values[0] < knockInPrice:
           isKnockIn = True
        # 判断是否是观察日
        if simulateDate in observeList:
            
        
        
        simulateDate = (datetime.datetime.strptime(simulateDate,'%Y%m%d') + relativedelta(days=1)).strftime('%Y%m%d')
        
        
    

In [85]:

for index, row in quote.iterrows():
    print(row)
    # 锚定日期
    anchorDate = row['trade_date']
    print(anchorDate)
    # 锚定价格
    anchorPrice = row['close']
    print(anchorPrice)
    # 敲出价格
    knockOutPrice = anchorPrice * BusinessVal.knockOutRatio
    print(knockOutPrice)
    # 敲入价格
    knockInPrice = anchorPrice * BusinessVal.knockInRatio
    print(knockInPrice)
    # 观察日列表
    observeList = []
    for i in range(1,BusinessVal.timeLimit + 1,1):
        observeList.append(getLeastTradeDate((datetime.datetime.strptime(anchorDate,'%Y%m%d') + relativedelta(months=i)).strftime('%Y%m%d')))
    print(observeList)
    simulate(anchorDate,anchorPrice,knockOutPrice,knockInPrice,observeList)
    break;

ts_code         000001.SH
trade_date       20100104
close             3243.76
open              3289.75
high              3295.28
low               3243.32
pre_close         3277.14
change            -33.379
pct_chg           -1.0185
vol           1.09448e+08
amount        1.33773e+08
Name: 2522, dtype: object
20100104
3243.76
3243.76
2595.0080000000003
['20100204', '20100304', '20100406', '20100504', '20100604', '20100705', '20100804', '20100906', '20101008', '20101104', '20101206', '20110104']
